In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd drive/MyDrive/Colab\ Notebooks/Synset_Grouping

/content/drive/.shortcut-targets-by-id/1GSBPdsf5hLXUWvkP6mXrY40FQGH44Er5/Synset_Grouping


In [ ]:
ls

1_prepare_synset_data.ipynb
2_prepare_synset_data_part2.ipynb
3_group_synset.ipynb
4_label_cluster_title_to_topic4.ipynb
cluster_matrix_topic4_5000sents.csv
preprocessed_topic4_corpus.csv
preprocessed_topic4_verb_list.csv
topic4_dataset.csv
topic4_sent_corpus.csv
topic4_verb_cluster_data_5000sents.csv
topic4_verb_list_5000sents.csv
vp_synset_data_topic4_5000sents.csv
wrong_4_label_cluster_title_to_topic4.ipynb


Created files: 
- 'vp_list.csv' : VP, Main Verb, Lemmatized Verb
- 'lemma_list.csv' : Lemmatized Verb, Frequency, Synset, Status

In [ ]:
import time

start_time = time.time()

In [ ]:
import spacy   
import numpy as np
import pandas as pd
from spacy.matcher import Matcher                                                                                                                                                                                         
nlp = spacy.load('en_core_web_sm')  # download model first
from nltk import tokenize

from itertools import chain
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

In [ ]:
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Unzipping corpora/omw-1.4.zip.


True

### Load dataset and tokenize

In [ ]:
vp_df = pd.read_csv('preprocessed_topic4_verb_list.csv')
vp_df.head()

,Unnamed: 0,doc_id,sent_id,verb_id,verb,check_verb,main_verb,lemma_verb
0,0,0,0,7,related,"[(9148364968459358513, 0, 1)]",related,relate
1,0,0,0,11,-related,"[(9148364968459358513, 0, 1)]",-related,-related
2,0,0,0,14,developing,"[(9148364968459358513, 0, 1)]",developing,develop
3,0,0,0,27,conducted,"[(9148364968459358513, 0, 1)]",conducted,conduct
4,1,0,1,2,including,"[(9148364968459358513, 0, 1)]",including,include


In [ ]:
# unique vp list with frequency and status
# status --> if there is no synsets for a verb, this might be a wrong verb (e.g. amd)-related), 
#           so if a verb doesn't have synsets, status = invalid, otherwise, status = valid

vp_freq_df = pd.DataFrame()
vp_freq_df = pd.Series(vp_df['lemma_verb']).value_counts().sort_index().reset_index().reset_index(drop=True)
vp_freq_df.columns = ['Lemmatized Verb', 'Frequency']

vp_freq_df.head()

,Lemmatized Verb,Frequency
0,-1130,2
1,-2665,1
2,-adjusted,1
3,-appearing,1
4,-approved,1


In [ ]:
vp_synset_df = pd.DataFrame()
vp_synset_df['Lemmatized Verb'] = ""
vp_synset_df['Frequency'] = ""
vp_synset_df['Synset'] = ""
vp_synset_df['Status'] = ""

In [ ]:
for idx in range(len(vp_freq_df)):
    
    lemma_mv = vp_freq_df['Lemmatized Verb'][idx]
    vp_synset_df.loc[idx,'Lemmatized Verb'] = lemma_mv
    vp_synset_df.loc[idx,'Frequency'] = vp_freq_df['Frequency'][idx]
    
    # get all verb synset of mv 
    mv_syn = wordnet.synsets(lemma_mv, pos='v')
    
    # get lemma set of each synset of mv - all synonyms of mv
    mv_syn_lemma = [mv_syn[i].lemma_names() for i in range(len(mv_syn))]
    mv_syn_lemma = list(chain.from_iterable(mv_syn_lemma))
    mv_syn_lemma = list(set(mv_syn_lemma))
    #print('\nmv_syn_lemma: ',mv_syn_lemma)
    
    if mv_syn_lemma:
        vp_synset_df.loc[idx,'Synset'] = np.array(mv_syn_lemma, dtype=object)
        vp_synset_df.loc[idx,'Status'] = 'valid'
    else:
        vp_synset_df.loc[idx,'Status'] = 'invalid'

In [ ]:
vp_synset_df.head()

,Lemmatized Verb,Frequency,Synset,Status
0,-1130,2,NaN,invalid
1,-2665,1,NaN,invalid
2,-adjusted,1,NaN,invalid
3,-appearing,1,NaN,invalid
4,-approved,1,NaN,invalid


In [ ]:
vp_synset_df[vp_synset_df['Status'] == 'invalid']

NameError: ignored

In [ ]:
vp_synset_df.to_csv('preprocessed_topic4_vp_synset.csv')

In [ ]:
end_time = time.time()

print('Estimate runtime (total): ', (end_time-start_time)/60, ' minutes.')

Estimate runtime (total):  0.2910415252049764  minutes.


In [ ]:
# Estimate runtime (total):  0.2910415252049764  minutes.